In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
# import utils
import openai
from dotenv import load_dotenv

load_dotenv()
print("hello")

hello


In [3]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [4]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import Document

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [5]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.core.schema.Document'>
Doc ID: 1362f3f5-f526-441d-bf38-ee093155f956
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


### Auto-merging Retrieval Setup

In [6]:
from llama_index.core import Document
document = Document(text="\n\n".join([doc.text for doc in documents]))

In [8]:
from llama_index.core.node_parser import HierarchicalNodeParser

#any decreasing order of chunk size
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048,512,128]
)

In [16]:
nodes = node_parser.get_nodes_from_documents([document])
nodes

[TextNode(id_='11a601b9-bc15-4e70-998b-d681a815dfb6', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='bea66d11-0adb-4b7f-91af-d817b33e6ea4', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='fdc7ce9a017ce62d4e0b30cc419051a9bb56618a8569f51a2b64f8535fe6244e'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='198ef407-22c3-417c-9689-4ba87bceb1cf', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='601594bb882270ad63b4547797595e35b471287cec74dcc07134629cbb7d71f5'), <NodeRelationship.CHILD: '5'>: [RelatedNodeInfo(node_id='fdc32283-d4b7-4a93-82a4-616f1aef29ee', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='2b4de016a7c855ab9278e03c30a440bb34e81a1ea68fd3225250abf084e52330'), RelatedNodeInfo(node_id='7c093177-af0f-42e4-840c-afa702938296', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='0c8c4015b5079720c9a99a811c9a8888c9f8ee3b9265eec5507fee681025f69c'), 

In [11]:
from llama_index.core.node_parser import get_leaf_nodes

leaf_nodes = get_leaf_nodes(nodes)
print(leaf_nodes[30].text)

But this became less important as numerical linear algebra libraries matured.
Deep learning is still an emerging technology, so when you train a neural network and the 
optimization algorithm struggles to converge, understanding the math behind gradient 
descent, momentum, and the Adam  optimization algorithm will help you make better decisions. 
Similarly, if your neural network does something funny — say, it makes bad predictions on 
images of a certain resolution, but not others — understanding the math behind neural network 
architectures puts you in a better position to figure out what to do.
Of course, I also encourage learning driven by curiosity.


In [12]:
nodes_by_id = {node.node_id: node for node in nodes}
parent_node = nodes_by_id[leaf_nodes[30].parent_node.node_id]
print(parent_node.text)

PAGE 12Should You 
Learn Math to 
Get a Job in AI? CHAPTER 3
LEARNING

PAGE 13Should you Learn Math to Get a Job in AI? CHAPTER 3
Is math a foundational skill for AI? It’s always nice to know more math! But there’s so much to 
learn that, realistically, it’s necessary to prioritize. Here’s how you might go about strengthening 
your math background.
To figure out what’s important to know, I find it useful to ask what you need to know to make 
the decisions required for the work you want to do. At DeepLearning.AI, we frequently ask, 
“What does someone need to know to accomplish their goals?” The goal might be building a 
machine learning model, architecting a system, or passing a job interview.
Understanding the math behind algorithms you use is often helpful, since it enables you to 
debug them. But the depth of knowledge that’s useful changes over time. As machine learning 
techniques mature and become more reliable and turnkey, they require less debugging, and a 
shallower understand

### Building the index

In [14]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [15]:
from llama_index.core import ServiceContext

auto_merging = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    node_parser=node_parser
)

In [17]:
from llama_index.core import VectorStoreIndex,StorageContext

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

automerging_index = VectorStoreIndex(
    leaf_nodes,storage_context=storage_context,service_context=auto_merging
)
automerging_index.storage_context.persist(persist_dir="./merging_index")

### Defining Retriever and Query Engine

In [18]:
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

automerging_retriever = automerging_index.as_retriever(
    similarity_top_k = 12 
)

retriever = AutoMergingRetriever(
    automerging_retriever,
    automerging_index.storage_context,
    verbose=True
)

In [21]:
rerank = SentenceTransformerRerank(top_n=6,model="BAAI/bge-reranker-base")

auto_merging_engine = RetrieverQueryEngine.from_args(
    automerging_retriever,
    node_postprocessors=[rerank]
)

In [22]:
auto_merging_response = auto_merging_engine.query(
    'What is the importance of networking in AI?'
)

In [23]:
from llama_index.core.response.notebook_utils import display_response

display_response(auto_merging_response)

**`Final Response:`** Networking in AI is crucial as it allows individuals to build a strong professional community that can provide valuable information, support, and opportunities. By connecting with others in the field, individuals can receive guidance, advice, and potential referrals to employers. Additionally, networking helps individuals stay updated on industry trends, collaborate on projects, and develop new ideas through interactions with peers and mentors. Ultimately, having a robust network in AI can propel individuals forward in their careers and provide a sense of belonging within the community.

### Putting all together from the above

In [39]:
import os

from llama_index.core import (
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.core.node_parser import HierarchicalNodeParser
from llama_index.core.node_parser import get_leaf_nodes
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine


def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context, service_context=merging_context
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=6,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine

In [41]:
from llama_index.llms.openai import OpenAI

index = build_automerging_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./merging_index1",
)

In [42]:
query_engine = get_automerging_query_engine(index, similarity_top_k=6)

### Trulens Evaluation

In [43]:
from trulens_eval import Tru
Tru().reset_database()

#### Two Layers

In [44]:
auto_merging_index_0 = build_automerging_index(
    documents,
    llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_0",
    chunk_sizes=[1024,512],
)

In [45]:
auto_merging_engine_0 = get_automerging_query_engine(
    auto_merging_index_0,
    similarity_top_k=12,
    rerank_top_n=6,
)

In [46]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_0,
    app_id='app_0'
)

In [47]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [48]:
def run_evals(eval_questions,tru_recorder,query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [50]:
run_evals(eval_questions, tru_recorder, auto_merging_engine_0)

> Merging 1 nodes into parent node.
> Parent node id: deacff93-daa6-4c73-a116-a62d1594b736.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: bbb327c7-712a-4d3a-9131-a5e732a02f88.
> Parent node text: PAGE 6The rapid rise of AI has led to a rapid rise in AI jobs, and many people are building excit...

> Merging 1 nodes into parent node.
> Parent node id: 1a969296-bb63-4f30-ac76-fab64b2d7668.
> Parent node text: PAGE 36Keys to Building a Career in AI CHAPTER 10
Of all the steps in building a career, this 
on...

> Merging 1 nodes into parent node.
> Parent node id: 9d877ad2-cf77-4da2-8ce1-6512fdba588d.
> Parent node text: PAGE 35Keys to Building a Career in AI CHAPTER 10
The path to career success in AI is more comple...

> Merging 1 nodes into parent node.
> Parent node id: 02f31bb8-48d9-4968-ad7f-f463b0784fd1.
> Parent node text: PAGE 15One of the most important

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

> Merging 1 nodes into parent node.
> Parent node id: 9d877ad2-cf77-4da2-8ce1-6512fdba588d.
> Parent node text: PAGE 35Keys to Building a Career in AI CHAPTER 10
The path to career success in AI is more comple...

> Merging 1 nodes into parent node.
> Parent node id: 28b489f3-4497-4d32-a00a-ee5c135b240a.
> Parent node text: PAGE 16Determine milestones. Once you’ve deemed a project sufficiently 
valuable, the next step i...

> Merging 1 nodes into parent node.
> Parent node id: 3981748e-f0d0-48b0-9bd5-73d4f93f57f8.
> Parent node text: PAGE 33Choose who to work with. It’s tempting to take a position because of the projects you’ll w...

> Merging 1 nodes into parent node.
> Parent node id: 79f9fd6c-30b0-47df-91d5-55199e17ec40.
> Parent node text: PAGE 22Over the course of a career, you’re likely to work on projects in succession, each growing...

> Merging 1 nodes into parent node.
> Parent node id: 915940ef-b0fc-404d-89b1-48ba3b5ba250.
> Parent node text: PAGE 38Before we dive into the f

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


...

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

> Merging 1 nodes into parent node.
> Parent node id: 915940ef-b0fc-404d-89b1-48ba3b5ba250.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...

> Merging 1 nodes into parent node.
> Parent node id: 5aea48b9-a5ff-41fc-9459-c6a7d4c3cfa5.
> Parent node text: PAGE 13Should you Learn Math to Get a Job in AI? CHAPTER 3
Is math a foundational skill for AI? I...

> Merging 1 nodes into parent node.
> Parent node id: b3f1f136-3d6c-4f9a-bbb9-1fd5e5baffc8.
> Parent node text: PAGE 7These phases apply in a wide 
range of professions, but AI 
involves unique elements.
For e...

> Merging 1 nodes into parent node.
> Parent node id: a35ad641-d9e8-4af4-9b49-ae5cfab9f918.
> Parent node text: PAGE 18It goes without saying that we should only work on projects that are responsible, ethical,...

> Merging 1 nodes into parent node.
> Parent node id: 8cbcdfdb-199e-42df-8fea-7266193918ee.
> Parent node text: PAGE 10This is a lot to learn!
E

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

In [51]:
Tru().get_leaderboard(app_ids=[])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
app_0,0.927083,0.381667,0.89,11.8,0.003578


#### Three Layers

In [57]:
auto_merging_index_1 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_1",
    chunk_sizes=[2048,512,128],
)

In [58]:
auto_merging_engine_1 = get_automerging_query_engine(
    auto_merging_index_1,
    similarity_top_k=12,
    rerank_top_n=6,
)

In [59]:
tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_1,
    app_id ='app_1'
)

In [60]:
run_evals(eval_questions, tru_recorder, auto_merging_engine_1)

> Merging 2 nodes into parent node.
> Parent node id: 34090762-68bb-4404-906a-354e560f5645.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 1194edeb-5d9c-4305-8b2a-85395370abd3.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...



Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

> Merging 3 nodes into parent node.
> Parent node id: b44bd146-80ef-4669-a8d8-64be4475c99f.
> Parent node text: PAGE 35Keys to Building a Career in AI CHAPTER 10
The path to career success in AI is more comple...

> Merging 1 nodes into parent node.
> Parent node id: f0f67e46-1533-40bf-b848-939cf5a74ddb.
> Parent node text: PAGE 35Keys to Building a Career in AI CHAPTER 10
The path to career success in AI is more comple...



Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

...

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

> Merging 3 nodes into parent node.
> Parent node id: cc59cd85-d66e-45c5-a163-055d9fdc83bd.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...

> Merging 1 nodes into parent node.
> Parent node id: 31cd681f-9491-411c-a0d5-71b1b46696a6.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...



Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

In [61]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
app_0,0.927083,0.381667,0.890,11.8,0.003578
app_1,0.819444,0.428333,0.885,9.3,0.001568


In [62]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.0.103:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>